## Code to check GPU integration


In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("Enabled memory growth for GPUs:", gpus)
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPUs found")


2025-11-11 12:28:59.468840: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 12:29:00.340294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-11 12:29:03.564782: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Enabled memory growth for GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Imports


In [2]:
import pathlib, os
from pathlib import Path
import tensorflow as tf
from collections import Counter
import numpy as np
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
import datetime

# --- NEW IMPORTS FOR TRANSFER LEARNING ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

## Constants

In [3]:
DATA_DIR = str("asl_alphabet_train")
IMG_SIZE = (200, 200)
BATCH_SIZE = 32
SEED = 42
VAL_SPLIT = 0.15

## Dataset imports

In [4]:
# training dataset (subset="training") and validation dataset (subset="validation")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

Found 87000 files belonging to 29 classes.
Using 73950 files for training.


I0000 00:00:1762864152.819160  799113 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4950 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 87000 files belonging to 29 classes.
Using 13050 files for validation.


## Class checks

In [5]:
class_names = train_ds.class_names
print("Found classes:", len(class_names))
print(class_names)

Found classes: 29
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [6]:
from collections import Counter
p = pathlib.Path(DATA_DIR)
counts = {d.name: len(list(d.glob("*"))) for d in p.iterdir() if d.is_dir()}
for c, n in sorted(counts.items()):
    print(f"{c:20s} {n}")


A                    3000
B                    3000
C                    3000
D                    3000
E                    3000
F                    3000
G                    3000
H                    3000
I                    3000
J                    3000
K                    3000
L                    3000
M                    3000
N                    3000
O                    3000
P                    3000
Q                    3000
R                    3000
S                    3000
T                    3000
U                    3000
V                    3000
W                    3000
X                    3000
Y                    3000
Z                    3000
del                  3000
nothing              3000
space                3000


In [7]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [8]:
iterator = iter(train_ds)

first_batch = next(iterator)

images_batch, labels_batch = first_batch


print(f"Type of images_batch: {type(images_batch)}")
print(f"Images batch shape: {images_batch.shape}")
print(f"Labels batch shape: {labels_batch.shape}")

print("\n--- Example from the batch ---")
print(f"Shape of one image: {images_batch[0].shape}")
print(f"Label for first image: {labels_batch[0]}")

Type of images_batch: <class 'tensorflow.python.framework.ops.EagerTensor'>
Images batch shape: (32, 200, 200, 3)
Labels batch shape: (32,)

--- Example from the batch ---
Shape of one image: (200, 200, 3)
Label for first image: 25


## Optimize pipeline (for Transfer Learning)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

# We can use the same augmentation as before
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),  
    tf.keras.layers.RandomZoom(0.1),    
    tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), 
    tf.keras.layers.RandomBrightness(0.1), 
    tf.keras.layers.RandomContrast(0.1)    
])

# --- Apply augmentation, THEN MobileNet's specific pre-processing ---
# This function scales pixels from [0, 255] to [-1, 1]
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

# --- Apply pre-processing to validation data ---
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [10]:
num_classes = len(class_names)
input_shape = (*IMG_SIZE, 3) 

# 1. Load the "Street Smart" Base Model (MobileNetV2)
# We don't include its final "top" layer
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False, 
    weights='imagenet' # Load the knowledge from millions of images
)

# 2. FREEZE the expert's knowledge. 
# We don't want to re-train the part that knows about edges and textures.
base_model.trainable = False

# 3. Build your new model on top
model = models.Sequential([
    # Start with the frozen expert
    base_model,
    
    # Add our own classifier head
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3), # A dropout of 0.3 is a good starting point
    layers.Dense(num_classes, activation='softmax') # Your 29-class output
])

model.summary()

/tmp/ipykernel_799113/1955976156.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 5s 1us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,928,733 (11.17 MB)

 Trainable params: 670,749 (2.56 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # <-- Perfect for integer labels!
    metrics=['accuracy']
)

In [12]:
# Create a log directory for this specific run
# The datetime string makes each run unique in TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the TensorBoard callback
tensorboard_callback = callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1  # This logs weight histograms every epoch
)

In [13]:
# Create a callback to stop training if validation accuracy doesn't improve
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3, # Stop after 3 epochs of no improvement
    restore_best_weights=True # Automatically restore the best model weights
)

# Create a callback to save your best model to a file
model_checkpoint = callbacks.ModelCheckpoint(
    'model1.keras', # File name
    monitor='val_accuracy',
    save_best_only=True
)

In [14]:
EPOCHS = 20 # Start with 20, EarlyStopping will stop it if it's done sooner

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping,
     model_checkpoint,
     tensorboard_callback
     ] # Pass in our helpers
)

Epoch 1/20


2025-11-11 12:29:31.335992: I external/local_xla/xla/service/service.cc:163] XLA service 0x76f4c4002c00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-11 12:29:31.336037: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-11-11 12:29:31.693912: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-11 12:29:33.138685: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91500
2025-11-11 12:29:33.340270: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-11 12:29:33.

2309/2311 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4702 - loss: 1.8103

2025-11-11 12:31:01.423675: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-11 12:31:03.022824: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4437', 16 bytes spill stores, 16 bytes spill loads

2025-11-11 12:31:03.169636: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4437', 288 bytes spill stores, 288 bytes spill loads

2025-11-11 12:31:10.220061: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng3{k11=0} for conv (f32[30,576,13,13]{3,2,1,0}, u8[0]{0}) custom-call(f32[30,576,13,13]{3,

2311/2311 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4704 - loss: 1.8099

...

2310/2311 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8252 - loss: 0.5354

2025-11-11 12:53:53.449993: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 15360256 bytes after encountering the first element of size 15360256 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2311/2311 ━━━━━━━━━━━━━━━━━━━━ 85s 36ms/step - accuracy: 0.8269 - loss: 0.5276 - val_accuracy: 0.9266 - val_loss: 0.2084
Epoch 18/20
2311/2311 ━━━━━━━━━━━━━━━━━━━━ 85s 36ms/step - accuracy: 0.8281 - loss: 0.5233 - val_accuracy: 0.8998 - val_loss: 0.2985
Epoch 19/20
2311/2311 ━━━━━━━━━━━━━━━━━━━━ 87s 38ms/step - accuracy: 0.8301 - loss: 0.5163 - val_accuracy: 0.9267 - val_loss: 0.2157
Epoch 20/20
   2/2311 ━━━━━━━━━━━━━━━━━━━━ 3:17 85ms/step - accuracy: 0.8125 - loss: 0.4995  

2025-11-11 12:56:55.641874: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 15360256 bytes after encountering the first element of size 15360256 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2311/2311 ━━━━━━━━━━━━━━━━━━━━ 83s 36ms/step - accuracy: 0.8308 - loss: 0.5150 - val_accuracy: 0.9288 - val_loss: 0.2051
